In [1]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath('../'))
from src.hopsworks_connections import pull_data, upload_data
import pandas_ta as ta
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, precision_score
import numpy as np
import re

# Print the current working directory
current_working_directory = os.getcwd()
print("Current Working Directory:", current_working_directory)

/Users/davydsadovskyy/GitBackedShit/crypto-prediction/venv_crypto_prediction/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Current Working Directory: /Users/davydsadovskyy/GitBackedShit/crypto-prediction/notebooks


In [1]:
predictors = []

horizons = [2, 3, 4, 5, 10, 25, 50, 100]
for horizon in horizons:
    
    for indicator in ['ema', 'rsi', 'sma']:
        predictors.append(f'{indicator}_{horizon}')
    
    for indicator in ['ema', 'rsi', 'sma']:
        predictors.append(f'volume_{indicator}_{horizon}')

    # predictors.append(f'last_{horizon}_day_15th_pct_price_change_count')
    predictors.append(f'last_{horizon}_day_85th_pct_price_change_count')

    # predictors.append(f'last_{horizon}_day_15th_pct_volume_change_count')
    predictors.append(f'last_{horizon}_day_85th_pct_volume_change_count')
    
    # predictors.append(f'last_{horizon}_day_15th_pct_spread_count')
    predictors.append(f'last_{horizon}_day_85th_pct_spread_count')

months = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
predictors.extend(months)
weekdays = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
predictors.extend(weekdays)


# model = RandomForestClassifier(n_estimators=250, min_samples_split=200, random_state=1)
model = cb.CatBoostClassifier(iterations=300, learning_rate=.01, depth=6, verbose=False, random_seed=1)

NameError: name 'cb' is not defined

In [ ]:
# model = LogisticRegression(penalty='l1', C=.1, solver='liblinear', random_state=1)  # error, failed to converge,
model = LogisticRegression(penalty='l1', C=.1, solver='liblinear', random_state=1, max_iter=1000)

df = get_backtested_predictions_df(eth_ohlc, predictors, model, "tmw_1_percent_increase_binary", start=1000, step=100)
df['prediction'] = df['probability'] >= 0.55
df = df[:-1]
print(precision_score(df['tmw_1_percent_increase_binary'], df['prediction']))
df['prediction'].value_counts()

### Get raw ethereum ohlc data

In [2]:
def get_cleaned_raw_data(filename):
    eth_ohlc_raw = pd.read_csv(f'../data/raw/ohlc/{filename}', parse_dates=True)
    def convert_value(value):
        """
        Converts a string value to a float. Removes $ signs, and converts
        billion (bn), million (m), and thousand (K) values to their numeric equivalents.
        """
        value = value.replace('$', '')  # Remove $ sign to simplify processing
        if value[-1].lower() == 'm':
            return float(value[:-1]) * 1_000_000
        elif value[-1].lower() == 'b':
            return float(value[:-1]) * 1_000_000_000
        elif value[-1].lower() == 'k':
            return float(value[:-1]) * 1_000
        elif value[-2:].lower() == 'bn':  # Handle 'bn' for billions
            return float(value[:-2]) * 1_000_000_000
        else:
            return float(value)

    eth_ohlc_raw.columns = [col.lower() for col in eth_ohlc_raw.columns]
    eth_ohlc_raw.rename(columns={'volume(eth)': 'volume_eth', 'market cap': 'market_cap'}, inplace=True)

    for col in ['open', 'high', 'low', 'close', 'volume', 'market_cap']:
        eth_ohlc_raw[col] = eth_ohlc_raw[col].apply(convert_value)
    eth_ohlc_raw['date'] = pd.to_datetime(eth_ohlc_raw['date'])

    return eth_ohlc_raw

In [3]:
df_raw = get_cleaned_raw_data('ethereum.csv')
df_raw = df_raw.iloc[::-1].reset_index(drop=True) # reverse order
df_raw

,date,open,high,low,close,volume,volume_eth,market_cap
0,2015-08-07,2.8300,3.5400,2.5200,2.7700,1.643000e+05,56374.0,1.753000e+08
1,2015-08-08,2.7900,2.8000,0.7147,0.7533,6.742000e+05,382082.0,1.065000e+08
2,2015-08-09,0.7061,0.8798,0.6292,0.7019,5.322000e+05,729741.0,4.410000e+07
3,2015-08-10,0.7140,0.7299,0.6365,0.7084,4.053000e+05,581293.0,4.210000e+07
4,2015-08-11,0.7081,1.1300,0.6632,1.0700,1.500000e+06,1638721.0,5.390000e+07
...,...,...,...,...,...,...,...,...
3157,2024-04-17,3083.0000,3119.0000,2926.0000,2984.0000,1.550000e+10,5093979.0,3.722000e+11
3158,2024-04-18,2986.0000,3087.0000,2960.0000,3064.0000,1.350000e+10,4453302.0,3.706000e+11
3159,2024-04-19,3061.0000,3117.0000,2879.0000,3085.0000,1.740000e+10,5675824.0,3.746000e+11
3160,2024-04-20,3085.0000,3166.0000,3025.0000,3155.0000,8.900000e+09,2884086.0,3.776000e+11


In [111]:
df_with_response_and_preds = add_predictors(df_with_response_vars, horizons_list=[2,3,4,5,6,7,10,14,21,25,30,50,75,100])
len(df_with_response_and_preds)

3162

In [112]:
# Drop NA values
df_with_response_and_preds = df_with_response_and_preds.dropna()
len(df_with_response_and_preds) # rows dropped = length of longest horizon + last row which has price target for tomorrow

3061

### Get set of predictors that have low correlation

In [116]:
predictors = []
periods = [2, 5, 10, 25, 50, 100]
percent_increase_counts = ['1_0', '1_25', '1_5', '1_75', '2_0', '2_25', '2_5', '2_75', '3_0']
for period in periods:
    # for indicator in ['ema', 'rsi', 'sma', 'vma_ema', 'vma_sma', 'roc_vol']:
    for indicator in ['ema', 'rsi', 'sma']:
        predictors.append(f'{indicator}_{period}')
    for percent in percent_increase_counts:
        predictors.append(f'last_{period}_day_{percent}_percent_increase_count')
months = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
predictors.extend(months)
weekdays = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
predictors.extend(weekdays)
# other_jawns = ['vol_osc_10_25', 'vwap', 'adl']
# predictors.extend(other_jawns)
len(predictors)

91

In [105]:
# returns a list of noncorrelated predictors
def drop_predictors_with_high_correlation(data, predictors_list, correlation_threshold):

    corr_matrix = data[predictors_list].corr()

    # Threshold for high correlation
    high_corr_threshold = correlation_threshold

    high_corr_pairs = []
    for col in corr_matrix.columns:
        for row in corr_matrix.index:
            if col != row and corr_matrix.at[row, col] > high_corr_threshold:
                # Sort the pair to avoid duplicates like (A, B) and (B, A)
                pair = tuple(sorted([row, col]))
                if pair not in high_corr_pairs:
                    high_corr_pairs.append(pair)

    # Identify and remove correlation pairs if the second element occurs as a first element in the list of all pairs
    first_elements = [pair[0] for pair in high_corr_pairs]
    filtered_pairs = [pair for pair in high_corr_pairs if pair[1] not in first_elements]

    # Find the unique first elements of all the pairs
    predictors_high_cor = list(set(pair[0] for pair in filtered_pairs))

    print("original number of predictors in model: ", len(predictors))
    print("number of unique highly correlated predictors to be dropped from model: ", len(predictors_high_cor))

    # Removing elements from predictors that are in predictors_high_cor
    predictors_low_cor = [x for x in predictors if x not in predictors_high_cor]
    len(predictors_low_cor)

    print("number of predictors remaining in model: ", len(predictors_low_cor))

    return predictors_low_cor

In [117]:
predictors_low_cor = drop_predictors_with_high_correlation(df_with_response_and_preds, predictors, 0.96)
print('\n', predictors_low_cor)

original number of predictors in model:  91
number of unique highly correlated predictors to be dropped from model:  13
number of predictors remaining in model:  78

 ['ema_2', 'rsi_2', 'sma_2', 'last_2_day_1_0_percent_increase_count', 'last_2_day_1_25_percent_increase_count', 'last_2_day_1_5_percent_increase_count', 'last_2_day_1_75_percent_increase_count', 'last_2_day_2_0_percent_increase_count', 'last_2_day_2_25_percent_increase_count', 'last_2_day_2_5_percent_increase_count', 'last_2_day_2_75_percent_increase_count', 'last_2_day_3_0_percent_increase_count', 'rsi_5', 'sma_5', 'last_5_day_1_0_percent_increase_count', 'last_5_day_1_25_percent_increase_count', 'last_5_day_1_5_percent_increase_count', 'last_5_day_1_75_percent_increase_count', 'last_5_day_2_25_percent_increase_count', 'last_5_day_2_5_percent_increase_count', 'last_5_day_3_0_percent_increase_count', 'rsi_10', 'sma_10', 'last_10_day_1_0_percent_increase_count', 'last_10_day_1_25_percent_increase_count', 'last_10_day_1_5_pe

## Train Model

In [31]:
def get_precision_score(data, model, predictors, response_var, threshold, start=2800, step=50):
    
    def predict(train, test, predictors, model):
        model.fit(train[predictors], train[response_var])
        probability = model.predict_proba(test[predictors])[:,1]
        proba_series = pd.Series(probability, index=test.index, name="probability")
        combined = pd.concat([test[response_var], test['tmw_percent_increase'], proba_series], axis=1)
        return combined

    all_predictions = []
    for i in range(start, data.shape[0], step):
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i+100)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)

    data_with_predictions = pd.concat(all_predictions)

    col_name = f'pred_{threshold}'
    data_with_predictions[col_name] = data_with_predictions['probability'] >= threshold

    ppv = precision_score(data_with_predictions[response_var], data_with_predictions[col_name])
    
    return ppv

In [109]:
predictors = []
periods = [2, 5, 10, 25, 50, 100]
percent_increase_counts = ['1_0', '1_25', '1_5', '1_75', '2_0', '2_25', '2_5', '2_75', '3_0']
for period in periods:
    for indicator in ['ema', 'rsi', 'sma', 'vma_ema', 'vma_sma', 'roc_vol']:
        predictors.append(f'{indicator}_{period}')
    for percent in percent_increase_counts:
        predictors.append(f'last_{period}_day_{percent}_percent_increase_count')
months = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
predictors.extend(months)
weekdays = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
other_jawns = ['vol_osc_10_25', 'vwap', 'adl']
predictors.extend(weekdays)
len(predictors)

109

In [119]:
model = RandomForestClassifier(n_estimators=250, min_samples_split=200, random_state=1)
response_var = 'tmw_1_0_percent_increase_binary'
threshold = 0.55

# 178 predictors... ppv = 0.5655737704918032
# 134 predictors (low corr ones < 0.96)....ppv = 0.5614035087719298
# 91 predictors (only price ones..) .... ppv = 0.5892857142857143
# 78 predictors (low corr < 0.96)..... ppv = 0.624
ppv = get_precision_score(df_with_response_and_preds, model, predictors_low_cor, response_var, threshold, start=2000, step=100)
ppv

0.624

In [120]:
df_with_response_and_preds['tmw_1_0_percent_increase_binary'][2000:].value_counts(normalize=True)

tmw_1_0_percent_increase_binary
0    0.552309
1    0.447691
Name: proportion, dtype: float64

In [ ]:
def plot_ppv_precision_responses_corr_predictors():

In [123]:
def get_backtested_predictions_df(data, predictors, model, response_var, start=2000, step=100):
    def predict(train, test, predictors, model):
        model.fit(train[predictors], train[response_var])
        probability = model.predict_proba(test[predictors])[:,1]
        proba_series = pd.Series(probability, index=test.index, name="probability")
        combined = pd.concat([test[response_var], test['tmw_percent_increase'], proba_series], axis=1)
        return combined

    all_predictions = []
    for i in range(start, data.shape[0], step):
        train = data.iloc[1000:i].copy()
        test = data.iloc[i:(i+step)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)

    data_with_predictions = pd.concat(all_predictions)
    
    return data_with_predictions

In [122]:
list_of_dfs = []
intervals = [1 + i * 0.25 for i in range(int((3 - 1) / 0.25) + 1)] # 1-3, by .25
for i in intervals:
    df = get_backtested_predictions_df(df_with_response_and_preds, predictors_low_cor, model, f"tmw_{str(i).replace('.', '_')}_percent_increase_binary")
    list_of_dfs.append(df)

data len 3061
data len 3061
data len 3061
data len 3061
data len 3061
data len 3061
data len 3061
data len 3061
data len 3061


In [66]:
list_of_dfs_by_predictors_used = []
for corr_threshold in np.arange(0.5, 1.0, 0.01):
    predictors_low_cor = drop_predictors_with_high_correlation(df_with_response_and_preds, predictors, corr_threshold)
    df = get_backtested_predictions_df(df_with_response_and_preds, predictors_low_cor, model, f"tmw_1_0_percent_increase_binary")
    print(f'added corr_threshold={corr_threshold} model results')
    list_of_dfs_by_predictors_used.append((df, corr_threshold))

original number of predictors in model:  109
number of unique highly correlated predictors to be dropped from model:  42
number of predictors remaining in model:  67
data len 3061
added corr_threshold=0.5 model results
original number of predictors in model:  109
number of unique highly correlated predictors to be dropped from model:  42
number of predictors remaining in model:  67
data len 3061
added corr_threshold=0.51 model results
original number of predictors in model:  109
number of unique highly correlated predictors to be dropped from model:  41
number of predictors remaining in model:  68
data len 3061
added corr_threshold=0.52 model results
original number of predictors in model:  109
number of unique highly correlated predictors to be dropped from model:  41
number of predictors remaining in model:  68
data len 3061
added corr_threshold=0.53 model results
original number of predictors in model:  109
number of unique highly correlated predictors to be dropped from model:  35


In [128]:
def plot_ppv_by_threshold(list_of_dfs, use_over_0_percent_target, min_true_count, threshold_start, threshold_end, threshold_step):
    # Initialize a Plotly figure
    fig = go.Figure()

    # Loop through each DataFrame and interval
    for df in list_of_dfs:
        filtered_thresholds = []
        filtered_ppvs = []
        filtered_hover_texts = []  # List to store hover text information only for true_count >= min_true_count

        # Use regular expressions to extract the number or word describing the interval
        first_column_name = df.columns[0]
        match = re.search(r'tmw_([\d_]+|\w+)_percent_increase_binary', first_column_name)
        if match:
            interval = match.group(1).replace('_', '.')  # Replace underscores with dots for floating point values
            if interval == 'positive':
                interval_label = 'Positive'
            else:
                interval_label = f'{interval}'
                
        thresholds = [x * threshold_step for x in range(int(threshold_start / threshold_step), int(threshold_end / threshold_step) + 1)]
        for threshold in thresholds:
            col_name = f'pred_{threshold}'
            df[col_name] = df['probability'] >= (threshold / 100)
            
            # Calculate the confusion matrix
            # response_col_name = f"tmw_{str(interval).replace('.', '_')}_percent_increase_binary"
            if use_over_0_percent_target:
                cm = confusion_matrix(df['tmw_positive_percent_increase_binary'], df[col_name])
            else:
                response_col_name = df.columns[0]
                cm = confusion_matrix(df[response_col_name], df[col_name])

            tn, fp, fn, tp = cm.ravel()
            ppv = tp / (tp + fp) if (tp + fp) > 0 else 0

            # Calculate value counts for hover information
            value_counts = df[col_name].value_counts().to_dict()
            false_count = value_counts.get(False, 0)
            true_count = value_counts.get(True, 0)
            
            # Only include data points where true_count >= min_true_count
            if true_count >= min_true_count:
                filtered_thresholds.append(threshold)
                filtered_ppvs.append(ppv)
                hover_text = f"{interval}% Return Tomorrow<br>Threshold: {threshold}<br>PPV: {ppv:.4f}<br>0: {false_count}<br>1: {true_count}"
                filtered_hover_texts.append(hover_text)

        # Add a trace for each DataFrame, only if there are valid data points
        if filtered_ppvs:
            fig.add_trace(go.Scatter(
                x=filtered_thresholds,
                y=filtered_ppvs,
                mode='lines+markers',
                name=f'{interval_label}% Return Tomorrow',  # Using a dynamically generated name based on interval
                text=filtered_hover_texts,
                hoverinfo="text"
            ))

    # Update plot layout with a title for the legend
    fig.update_layout(
        title=f'Precision Score of Models Accross Different Threshold Ranges. (Filtered by sum(prediction=1) >= {min_true_count})',
        xaxis_title='Probability Threshold (%)',
        yaxis_title='Positive Predictive Value (PPV, Precision)',
        legend_title="Model With Response Variable",  # Setting the title of the legend
        hovermode="closest"  # Ensure hover-effect shows for the closest data point
    )

    # Show the plot
    fig.show()

In [74]:
def plot_ppv_by_threshold_and_corr_predictor_level(list_of_dfs, use_over_0_percent_target, min_true_count, threshold_start, threshold_end, threshold_step):
    # Initialize a Plotly figure
    fig = go.Figure()

    # Loop through each DataFrame and interval
    for df in list_of_dfs:
        filtered_thresholds = []
        filtered_ppvs = []
        filtered_hover_texts = []  # List to store hover text information only for true_count >= min_true_count
                
        thresholds = [x * threshold_step for x in range(int(threshold_start / threshold_step), int(threshold_end / threshold_step) + 1)]
        for threshold in thresholds:
            col_name = f'pred_{threshold}'
            df[0][col_name] = df[0]['probability'] >= (threshold / 100)
            
            # Calculate the confusion matrix
            # response_col_name = f"tmw_{str(interval).replace('.', '_')}_percent_increase_binary"
            if use_over_0_percent_target:
                cm = confusion_matrix(df[0]['tmw_percent_increase_binary'], df[0][col_name])
            else:
                response_col_name = df[0].columns[0]
                cm = confusion_matrix(df[0][response_col_name], df[0][col_name])

            tn, fp, fn, tp = cm.ravel()
            ppv = tp / (tp + fp) if (tp + fp) > 0 else 0

            # Calculate value counts for hover information
            value_counts = df[0][col_name].value_counts().to_dict()
            false_count = value_counts.get(False, 0)
            true_count = value_counts.get(True, 0)
            
            # Only include data points where true_count >= min_true_count
            if true_count >= min_true_count:
                filtered_thresholds.append(threshold)
                filtered_ppvs.append(ppv)
                hover_text = f"corr threshold = {df[1]:.2f}<br>Threshold: {threshold}<br>PPV: {ppv:.4f}<br>0: {false_count}<br>1: {true_count}"
                filtered_hover_texts.append(hover_text)

        # Add a trace for each DataFrame, only if there are valid data points
        if filtered_ppvs:
            fig.add_trace(go.Scatter(
                x=filtered_thresholds,
                y=filtered_ppvs,
                mode='lines+markers',
                name=f'corr={df[1]:.2f}',
                text=filtered_hover_texts,
                hoverinfo="text"
            ))

    # Update plot layout with a title for the legend
    fig.update_layout(
        title=f'Precision Score of Models Accross Different Threshold Ranges. (Filtered by sum(prediction=1) >= {min_true_count})',
        xaxis_title='Probability Threshold (%)',
        yaxis_title='Positive Predictive Value (PPV, Precision)',
        legend_title="Model With Different Num Predictors",  # Setting the title of the legend
        hovermode="closest"  # Ensure hover-effect shows for the closest data point
    )

    # Show the plot
    fig.show()

In [130]:
plot_ppv_by_threshold(list_of_dfs, True, 0, 50, 70, 0.25)

KeyError: 'tmw_positive_percent_increase_binary'

In [75]:
plot_ppv_by_threshold_and_corr_predictor_level(list_of_dfs_by_predictors_used, False, 0, 50, 70, 0.25)